<div class="alert alert-info">
    
An interactive version of this notebook is available on [Google Colab](https://colab.research.google.com/github/fpsim/fpsim/blob/main/docs/tutorials/T4_intervention_eligibility.ipynb?install=fpsim) or [Binder](https://mybinder.org/v2/gh/fpsim/fpsim/HEAD?labpath=docs%2Ftutorials%2FT4_intervention_eligibility.ipynb).
    
</div>

# Select a specific subpopulation for an intervention

At this point, you should now be able to run multiple simulations with distinct interventions, using default or custom parameters and plot the results. So far we've seen interventions that target a model parameter. Now we will see how we can we specify a function that tells FPsim who will receive the intervention. First, import the model and set your default settings.

In [ ]:
import fpsim as fp
import sciris as sc
import matplotlib.pyplot as plt

In [ ]:
pars = dict(
    n_agents   = 1_000,
    location   = 'kenya',
    start_year = 2000, 
    end_year   = 2012,
    exposure_factor = 1.0  # Overall scale factor on probability of becoming pregnant
)

Now, we will modify a 'state' value of all eligible women. To see all the states you can select based on, run this code:

In [ ]:
from fpsim.defaults import person_defaults
for i, state in enumerate(person_defaults): print(f"{i}: {state.name}")

## Increase the number of women who have education

The `change_people_state` intervention can be used to change 'states' or attributes of women. The states you can change are the same list of target states from above. Let's see what would hapen if we increase education. In FPsim, education is defined based on an education objective and education attainment. The education objective is given to each woman when she enters the model. During her school age years, she gains one year of educationl attainment each year until she reaches her objective. This trajectory can be interrupted if she experiences a pregnancy during those years. 

First, we will identify our eligible subpopulation for this intervention. In this case, we are selecting all women who have a goal to achieve at least one year of education.

In [ ]:
def select_undereducated(sim):
    """ Select women who have a goal to achieve at least 1 year of education """
    is_eligible = ((sim.people.female) &
                   (sim.people.alive)     & 
                   (sim.people.edu.objective > 0))
    return is_eligible

Now, define a baseline simulation, s0, with no changes.

In [ ]:
edu = fp.Education()
s0 = fp.Sim(pars=pars, education_module=edu, label='Baseline')

Next, we define our education change intervention. This intervention finds all of the eligible women (those with a goal of achieveing at least 1 year of education), and gives all of them 15 years of education, starting in the year 2010. We define a simulation, s1, with this educaiton intervention.

In [ ]:
change_education = fp.change_people_state(
                        'edu.attainment',
                        eligibility=select_undereducated,
                        years=2010.0,
                        new_val=15,  # Give all selected women 15 years of education
                    )
edu = fp.Education()
s1 = fp.Sim(pars=pars, 
            education_module=edu, 
            interventions=change_education,
            label='Increased education')

Run a multile simulation run with msim to get the results of both of these simulations.

In [ ]:
msim = fp.MultiSim(sims=[s0, s1])
msim.run(parallel=False, compute_stats=False)
s0, s1 = msim.sims

Plot the resulting education levels.

In [ ]:
plt.plot(s0.results.timevec, s0.results.edu.mean_attainment, label=s0.label)
plt.plot(s1.results.timevec, s1.results.edu.mean_attainment, label=s1.label)
plt.ylabel('Average years of education among women')
plt.xlabel('Year')
plt.legend()